In [ ]:
# Install dicom2nifti, intensity-nornmalization and ANTsPy

! pip install ../input/libraries/libEnv/dicom2nifti-2.3.0
! pip install ../input/libraries/libEnv/scikit-fuzzy-0.4.2
! pip install ../input/libraries/libEnv/intensity_normalization-1.4.5-py3-none-any.whl
! pip install ../input/libraries/libEnv/dipy-1.4.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl


In [ ]:
import numpy as np 
import pandas as pd 
import nibabel as nib
import SimpleITK as sitk
import nibabel.processing as nibproc
from dipy.io.image import load_nifti, save_nifti
# from intensity_normalization.normalize import zscore 
from dipy.align.imaffine import transform_centers_of_mass, AffineMap, MutualInformationMetric, AffineRegistration
from dipy.align.transforms import TranslationTransform3D, RigidTransform3D, AffineTransform3D

# Preprocessing fucntions 

def resampling_reorientation(img, output):    
    # resampling to 240x240x155 1mm3
    resampledIm = nibproc.conform(img, out_shape=(240, 240, 155), voxel_size=(1.0, 1.0, 1.0), order=1, cval=0.0)
    
    # Modify metadata for reorientation
    n1_header = img.header
    # print(n1_header)  
    n1_header.set_sform(np.diag([0, 0, 0, 0]), code='unknown')
    qform = np.array(
           [[ 1. ,   0. ,   0. , -120. ],
           [  0. ,   1. ,   0. , -129. ],
           [  0. ,   0. ,   1, -68.],
           [  0. ,   0. ,   0. ,   1. ]])

    n1_header.set_qform(qform, code='scanner')
    # print('--------------------------')
    # print(n1_header) 

    # Reoriented to SRI24 t1 Atlas 
    new_img = nib.nifti1.Nifti1Image(resampledIm.get_fdata(), None, header=n1_header)
    nib.save(new_img, output)

# Bias Correction
def N4BiasCorrection(imDir, output):
    # Read image define corrector
    image = sitk.ReadImage(imDir, sitk.sitkFloat32)
    print(type(image))
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    
    # Create mask
    mask = (image > 0)
    
    # Execute corrector
    corrector.SetMaximumNumberOfIterations(np.array([200], dtype='int').tolist())
    corrected_image = corrector.Execute(image, mask)
    print(type(corrected_image))

    sitk.WriteImage(corrected_image, output)
 

def rigidRegistration(static, moving, output):
    # Load static image
    static_data, static_affine, static_img = load_nifti(
                                                        static, 
                                                        return_img=True)
    static = static_data
    static_grid2world = static_affine

    # load moving image
    moving_data, moving_affine, moving_img = load_nifti(
                                                moving,
                                                return_img=True)
    moving = moving_data
    moving_grid2world = moving_affine

    # traslattion mass center
    c_of_mass = transform_centers_of_mass(static, static_grid2world,
                                           moving, moving_grid2world)
    transformed = c_of_mass.transform(moving)
    starting_affine = c_of_mass.affine

    # elements for registration
    nbins = 32
    sampling_prop = 5
    metric = MutualInformationMetric(nbins, sampling_prop)

    level_iters = [1]
    sigmas = [0.0]
    factors = [1]

    # Rigid registration 
    affreg = AffineRegistration(metric=metric,
                                level_iters=level_iters,
                                sigmas=sigmas,
                                factors=factors)

    transform = RigidTransform3D()
    params0 = None
    rigid = affreg.optimize(static, moving, transform, params0,
                            static_grid2world, moving_grid2world,
                            starting_affine = starting_affine)

    transformed = rigid.transform(moving)
    transformed = transformed.astype(np.uint16)
    save_nifti(output, transformed, static_affine)
    
    return rigid

# def z_norm(img, output):
#    zscore_normalize(img, mask=None)

In [ ]:
import dicom2nifti as dcmnii
import time
import os 

start_time = time.time()

# Conversion to nifti
pdata = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/test/'
casesList = os.listdir(pdata)
#casesList.sort()
caseTest = casesList[0]

pcase = pdata + caseTest

modalities = ["T2w", "T1wCE", "T1w", "FLAIR"]

pcaseo = '/kaggle/working/'+caseTest
os.mkdir(pcaseo) 

for modality in modalities: 
    pcaseMod = pcase+'/'+modality
    dcmnii.convert_directory(pcaseMod, pcaseo, compression=True)
    print('built '+pcaseo+'/'+modality)
    
# !tar -zcvf case.tar.gz /kaggle/working/00114

In [ ]:
# Preprocessing

modalities = ['T1C', 'T1', 'FLAIR', 'T2']

folder='/kaggle/working/00114/'
listFiles = os.listdir(folder)
listFiles.sort(reverse=True)
for modality in modalities:
    folderMod=folder+modality
    os.mkdir(folderMod)

for i in range(4):

    # Load image
    pdata = '/kaggle/working/00114/'+listFiles[i]
    img = nib.load(pdata)

    os.chdir('/kaggle/working/00114/'+modalities[i])

    # resampling to 240x240x155 1mm3
    resampling_reorientation(img, modalities[i]+'_resam&reor.nii.gz')

    # Bias correction
    N4BiasCorrection(modalities[i]+'_resam&reor.nii.gz', modalities[i]+'_bias.nii.gz')

    # Registration dipy
    if modalities[i] == 'T1C':
        static = '/kaggle/input/template/Template/T1_brain.nii'
        moving = modalities[i]+'_bias.nii.gz'
        output = modalities[i]+'_registered_SRI.nii.gz'
        rigid = rigidRegistration(static, moving, output)
    else:
        static = '/kaggle/working/00114/T1C/T1C_resam&reor.nii.gz'
        moving = modalities[i]+'_bias.nii.gz'
        output = modalities[i]+'_registered.nii.gz'
        rigidRegistration(static, moving, output)
 
modalities = ['T2', 'FLAIR', 'T1']

 # Load static image
static = '/kaggle/input/template/Template/T1_brain.nii'
static_data, static_affine, static_img = load_nifti(static, return_img=True)

for i in range(3):
   
    # load moving image
    os.chdir('/kaggle/working/00114/'+modalities[i])                                                          
    moving = modalities[i]+'_registered.nii.gz'
    moving_data, moving_affine, moving_img = load_nifti(moving, return_img=True)
    
    output = modalities[i]+'_registered_SRI.nii.gz'
    transformed = rigid.transform(moving_data)
    transformed = transformed.astype(np.uint16)

    save_nifti(output, transformed, static_affine)   
    
print("--- %s seconds ---" % (time.time() - start_time))

print('############ End Preprocessing ############')